## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [4]:
data = range(0,1000)
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [8]:
import pyspark
sc = pyspark.SparkContext('local[*]')


Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [9]:
rdd = sc.parallelize(data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.PipelinedRDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [10]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [11]:
# count
rdd.count()


1000

In [12]:
# first
rdd.first()

0

In [14]:
# take
rdd.take(10)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [15]:
# top
rdd.top(12)

[999, 998, 997, 996, 995, 994, 993, 992, 991, 990, 989, 988]

In [16]:
# collect
rdd.collect()

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [17]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([7.55023340e-01, 1.01914272e+00, 1.43333696e+00, 1.59656510e+00,
       8.31879512e-01, 4.52723138e+00, 2.65408421e+00, 5.54055043e+00,
       3.11130895e+00, 1.78437951e+00, 6.08013753e+00, 4.03092763e+00,
       1.10291941e+01, 3.50405708e+00, 8.65279353e+00, 1.33862601e+01,
       1.28910461e+01, 5.19049764e+00, 1.46882996e+01, 6.21295569e+00,
       2.65633227e+00, 1.92792466e+01, 7.36283507e+00, 3.70872397e+00,
       7.38998836e+00, 9.15960518e+00, 1.57594072e+01, 1.78544186e+01,
       2.34782963e+00, 1.51129458e+01, 1.80681333e+01, 3.36914791e+00,
       2.17505479e+01, 7.91353145e+00, 2.63194018e+01, 2.34275293e+01,
       9.86211004e+00, 3.31294055e+00, 3.53615251e+01, 7.88280977e+00,
       1.16902462e-01, 3.03495688e+01, 6.14466462e+00, 3.57715429e+01,
       6.41907873e+00, 1.02347820e+01, 3.59397570e+01, 5.63478942e+00,
       3.50783803e+01, 1.27931554e+01, 1.55593796e+01, 2.99009180e+01,
       1.75070287e+01, 2.71491801e+01, 4.45689704e+01, 5.44279541e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [19]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.take(4)


[0.7550233401856276,
 1.0191427173547274,
 1.4333369582876823,
 1.5965651001060466]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [21]:
def sales_tax(num):
    taxTot = num * .08
    return num - taxTot

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [22]:
# perform action to retrieve rdd values
revenue_minus_tax.take(10)

[0.6946214729707774,
 0.9376112999663492,
 1.3186700016246677,
 1.468839892097563,
 0.7653291514480508,
 4.165052865147347,
 2.441757474220873,
 5.097306397477676,
 2.862404233708676,
 1.6416291509256145]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [23]:
discounted = price_items.map(lambda x: x*.9)

In [24]:
discounted.take(10)

[0.6795210061670648,
 0.9172284456192547,
 1.2900032624589142,
 1.436908590095442,
 0.7486915611991802,
 4.074508237644143,
 2.3886757899986804,
 4.986495388836857,
 2.8001780547150092,
 1.6059415606881011]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [25]:
price_items.map(sales_tax).map(lambda x : x*0.9).top(15)


[781.8681560265247,
 776.5458232634664,
 730.336418652553,
 724.2152581432242,
 714.8061388658627,
 707.3599250627192,
 707.2496835061113,
 706.4628449695209,
 701.3494236922621,
 698.4767756974425,
 691.7472948829843,
 686.2801249125829,
 682.1087027996109,
 675.9464360015846,
 664.4446067800188]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [27]:
discounted.toDebugString()

b'(10) PythonRDD[11] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[6] at readRDDFromFile at PythonRDD.scala:262 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [28]:
mapped = price_items.map(lambda x: (x, x*0.92 *0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.7550233401856276, 0.6251593256736997), (1.0191427173547274, 0.8438501699697144), (1.4333369582876823, 1.186803001462201), (1.5965651001060466, 1.3219559028878067), (0.8318795124435335, 0.6887962363032458), (4.527231375160159, 3.7485475786326123), (2.6540842111096445, 2.197581726798786), (5.540550432040952, 4.587575757729908), (3.1113089496833437, 2.5761638103378086), (1.7843795118756678, 1.477466235833053)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [29]:
flat_mapped = price_items.flatMap(lambda x: (x, x*.92 * .9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.7550233401856276, 0.6251593256736997, 1.0191427173547274, 0.8438501699697144, 1.4333369582876823, 1.186803001462201, 1.5965651001060466, 1.3219559028878067, 0.8318795124435335, 0.6887962363032458]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [30]:
# use the filter function
selected_items = discounted.filter(lambda x: x>300)

# calculate total remaining in inventory 
selected_items.count()

310

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [31]:
selected_items.reduce(lambda x,y: x+y)

144931.2451181205

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [32]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(29, 302.7328770714878),
 (36, 331.08941168964367),
 (31, 315.305088244764),
 (38, 338.19948670570807),
 (30, 325.71165564802027),
 (9, 301.01569964631136),
 (29, 300.70177398204993)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [33]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x, y: x + y)
total_spent.take(10)

[(30, 3220.4388427962067),
 (20, 3285.5295555378943),
 (10, 714.4561850489172),
 (40, 4457.782578357703),
 (50, 3150.267130997971),
 (31, 3125.655561243314),
 (21, 3026.7084563886256),
 (1, 2572.283724006425),
 (41, 2501.990838678034),
 (11, 1121.1365012887486)]

In [34]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1],ascending = False).collect()

[(9, 7118.840166223609),
 (29, 5840.052804636802),
 (28, 5491.555785804291),
 (42, 5240.939138654442),
 (39, 5062.973422247257),
 (3, 5007.181543765433),
 (36, 4893.485557531116),
 (40, 4457.782578357703),
 (33, 4372.175070291609),
 (16, 4325.203329736933),
 (32, 3962.060504433691),
 (18, 3961.8233682848922),
 (38, 3811.99526766456),
 (47, 3494.884731496461),
 (8, 3297.0780812257444),
 (20, 3285.5295555378943),
 (30, 3220.4388427962067),
 (35, 3178.8085868112335),
 (50, 3150.267130997971),
 (31, 3125.655561243314),
 (12, 3091.569035556424),
 (21, 3026.7084563886256),
 (24, 2944.9072314483783),
 (7, 2938.312175201043),
 (15, 2904.582572240108),
 (4, 2770.217927739127),
 (1, 2572.283724006425),
 (44, 2570.0475546223524),
 (23, 2519.582783058173),
 (41, 2501.990838678034),
 (5, 2424.726327949018),
 (34, 2380.89574861059),
 (49, 2267.5075907002556),
 (2, 2133.1093629644756),
 (6, 2097.6933773620144),
 (22, 2071.316503194427),
 (14, 2028.3698041523921),
 (46, 2022.3220663798602),
 (45, 1934

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [35]:
total_items = sales_data.countByKey()
sorted(total_items.items(),key=lambda kv:kv[1],reverse=True)

[(9, 16),
 (29, 15),
 (42, 14),
 (28, 12),
 (3, 12),
 (36, 10),
 (18, 10),
 (39, 10),
 (31, 8),
 (38, 8),
 (30, 8),
 (21, 8),
 (16, 8),
 (33, 8),
 (20, 8),
 (40, 8),
 (50, 8),
 (8, 8),
 (32, 8),
 (35, 7),
 (4, 6),
 (44, 6),
 (47, 6),
 (41, 6),
 (23, 6),
 (7, 6),
 (24, 6),
 (12, 6),
 (34, 5),
 (14, 4),
 (45, 4),
 (1, 4),
 (26, 4),
 (2, 4),
 (46, 4),
 (5, 4),
 (49, 4),
 (22, 4),
 (15, 4),
 (6, 4),
 (13, 4),
 (17, 3),
 (43, 3),
 (48, 2),
 (10, 2),
 (19, 2),
 (25, 2),
 (11, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
